# Import thư viện

In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import regex as re
from nltk.corpus import stopwords
from keras.models import model_from_json
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Flatten, Dense, Softmax, Dropout
from sklearn.preprocessing import LabelEncoder
import math

In [44]:
movie = pd.read_csv('..\Info film\Movie\movie_info.csv')

In [45]:
movie.shape

(33838, 23)

In [46]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33838 entries, 0 to 33837
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Title                     33838 non-null  object 
 1   Tomatometer score         33838 non-null  int64  
 2   Audience score            30302 non-null  float64
 3   Tomatometer count         33838 non-null  object 
 4   Audience count            33838 non-null  object 
 5   Tomatometer state         33838 non-null  object 
 6   Audience state            30302 non-null  object 
 7   Genre                     33016 non-null  object 
 8   Original Language         32682 non-null  object 
 9   Director                  33678 non-null  object 
 10  Producer                  28287 non-null  object 
 11  Writer                    26653 non-null  object 
 12  Release Date (Theaters)   19694 non-null  object 
 13  Box Office (Gross USA)    13216 non-null  object 
 14  Runtim

In [47]:
movie = movie[['Title', 'Tomatometer score', 'Audience score', 'Tomatometer count',
                'Audience count', 'Genre', 'Original Language', 'Synopsis', 'Rating']]

In [48]:
movie.isnull().sum().sort_values()

Title                    0
Tomatometer score        0
Tomatometer count        0
Audience count           0
Genre                  822
Original Language     1156
Synopsis              1269
Audience score        3536
Rating               18171
dtype: int64

# Tiền xử lí dữ liệu

## Làm gọn cột "Rating"

In [49]:
for i in range(len(movie)):
    if pd.isna(movie['Rating'].iloc[i]):
        continue
    else:
        movie['Rating'].iloc[i] = movie['Rating'].iloc[i].split(' ')[0]

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_20220\2474650943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Rating'].iloc[i] = movie['Rating'].iloc[i].split(' ')[0]


## Drop các dòng mà cột "Synopsis" có giá trị Null

In [50]:
movie.dropna(axis=0, subset=['Synopsis'], inplace=True)

## Xử lí dữ liệu Null ở cột "Rating" và "Genre" bằng mô hình dự báo

In [51]:
# Hàm tiền xử lí dữ liệu text
def preprocess_text(text):
    text = text.lower() # Chuyển văn bản thành chữ thường
    text = re.sub(r'[^\w\s]',' ',text) # Xóa dấu câu
    text = re.sub("\d+", " ", text) # Xóa chữ số
    stop = stopwords.words('english') # Xóa các từ không có nghĩa (Stop Words)
    text = " ".join(text for text in text.split() if text not in stop)
    return text

In [52]:
def predict_info(movie_synopsis):
    t = vectorizer.transform([preprocess_text(movie_synopsis)])
    pre_class = np.argmax(loaded_model.predict(t.todense()), axis=1)
    return pre_class

### Dự báo "Rating"

In [53]:
rating = movie[['Rating', 'Synopsis']].copy().dropna().reset_index(drop=True)

In [54]:
# Tiền xử lí dữ liệu text trong cột "Synopsis" để đưa vào mô hình
for i in range(len(rating)):
    rating['Synopsis'].iloc[i] = preprocess_text(rating['Synopsis'].iloc[i])

In [55]:
# Load model predict Rating đã train
json_file = open('Predict Rating Model\predict_rating.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("Predict Rating Model\predict_rating.h5")

In [56]:
encoder = LabelEncoder()
labels = encoder.fit_transform(rating['Rating'].values)
rating['label'] = labels

In [57]:
x_train, x_test, y_train, y_test = train_test_split(rating['Synopsis'].values, rating['label'].values, test_size = 0.3, random_state = 0)

vectorizer = TfidfVectorizer()
vectorizer.fit_transform(x_train)

<10961x35933 sparse matrix of type '<class 'numpy.float64'>'
	with 367646 stored elements in Compressed Sparse Row format>

In [58]:
list_pre_rating = []

for syp in movie[movie['Rating'].isnull()]['Synopsis'].values:
    rating_class = predict_info(syp)
    rating = encoder.classes_[rating_class][0]
    list_pre_rating.append(rating)

movie.loc[movie['Rating'].isnull(), 'Rating'] = list_pre_rating

1/1 [==============================] - 0s 33ms/step


### Dự báo "Genre"

In [59]:
list_genre = []
for g in movie['Genre']:
    try:
        g = g.split(', ')
        for sub_g in g:
            if sub_g not in list_genre:
                list_genre.append(sub_g)
    except:
        continue

In [60]:
for g in list_genre:
    movie[g] = 0

for i in range(len(movie)):
    for g in list_genre:
        if not pd.isna(movie['Genre'].iloc[i]) and g in movie['Genre'].iloc[i]:
            movie[g].iloc[i] = 1

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_20220\645188401.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie[g].iloc[i] = 1


In [61]:
genre = movie[['Genre', 'Synopsis'] + list_genre].copy().dropna().reset_index(drop=True)

In [62]:
# Tiền xử lí dữ liệu text trong cột "Synopsis" để đưa vào mô hình
for i in range(len(genre)):
    genre['Synopsis'].iloc[i] = preprocess_text(genre['Synopsis'].iloc[i])

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_20220\819770565.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre['Synopsis'].iloc[i] = preprocess_text(genre['Synopsis'].iloc[i])


In [63]:
for gr in list_genre:
    print('Predict:', gr)
    # Chia bộ train và test (80:20)
    x_train, x_test, y_train, y_test = train_test_split(genre['Synopsis'].values, genre[gr].values, test_size = 0.2, random_state = 0)

    # Vectorizer
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(x_train)

    # Load model predict Genre đã train
    json_file = open('Predict Genre Model\predict_genre_%s.json'%gr, 'r')

    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights('Predict Genre Model\predict_genre_%s.h5'%gr)

    list_pre_rating = []

    for syp in movie[movie['Genre'].isnull()]['Synopsis'].values:
        t = vectorizer.transform([preprocess_text(syp)])
        pred = np.argmax(loaded_model.predict(t.todense()),axis=1)[0]
        list_pre_rating.append(int(pred))

    print(list_pre_rating)
    print('-'*150)
    movie.loc[movie['Genre'].isnull(), gr] = list_pre_rating

Predict: Drama
1/1 [==============================] - 0s 47ms/step
[1, 0, 0, 1, 0, 0, 0, 0, 1, 1]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Comedy
1/1 [==============================] - 0s 38ms/step
[0, 0, 1, 0, 1, 1, 0, 1, 1, 0]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Adventure
1/1 [==============================] - 0s 42ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------------------------------------------------------------------------------------------------------------------------------
Predict: Mystery & thriller
1/1 [==============================] - 0s 38ms/step
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
-----------------------------------------------------------------------------------------------------------------------------------------

### Drop cột "Genre"

In [64]:
movie.drop(columns=['Genre'], inplace=True)

## Drop các dòng trùng lặp

In [65]:
movie.duplicated().sum()

11

In [66]:
movie.drop_duplicates(inplace=True)

## Thay thế Null bằng giá trị xuất hiện nhiều nhất (mode) trong cột "Original Language" 

In [67]:
movie['Original Language'].fillna(movie['Original Language'].mode()[0], inplace=True)

In [68]:
movie.isnull().sum().sort_values()

Title                      0
Animation                  0
Lgbtq+                     0
Anime                      0
Music                      0
Holiday                    0
War                        0
Stand-up                   0
Nature                     0
History                    0
Sports & fitness           0
Sports                     0
Short                      0
Variety                    0
Foreign                    0
Special interest           0
Entertainment              0
News                       0
Other                      0
Faith & spirituality       0
Sci-fi                     0
Musical                    0
Western                    0
Tomatometer score          0
Tomatometer count          0
Audience count             0
Original Language          0
Synopsis                   0
Rating                     0
Drama                      0
Comedy                     0
Gay & lesbian              0
Adventure                  0
Action                     0
Romance       

## Thay thế Null ở cột "Audience score" bằng giá trị ở cột "Tomatometer score" tương ứng

In [69]:
movie['Audience score'].loc[movie['Audience score'].isnull()] = movie['Tomatometer score'].loc[movie['Audience score'].isnull()]

C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_20220\737363444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['Audience score'].loc[movie['Audience score'].isnull()] = movie['Tomatometer score'].loc[movie['Audience score'].isnull()]


In [70]:
movie.isnull().sum().sort_values()

Title                   0
Animation               0
Lgbtq+                  0
Anime                   0
Music                   0
Holiday                 0
War                     0
Stand-up                0
Nature                  0
History                 0
Sports & fitness        0
Sports                  0
Short                   0
Variety                 0
Foreign                 0
Special interest        0
Entertainment           0
News                    0
Other                   0
Faith & spirituality    0
Sci-fi                  0
Musical                 0
Western                 0
Tomatometer score       0
Audience score          0
Tomatometer count       0
Audience count          0
Original Language       0
Synopsis                0
Rating                  0
Drama                   0
Comedy                  0
Adventure               0
Mystery & thriller      0
Action                  0
Romance                 0
Crime                   0
Documentary             0
Horror      

## Tính điểm của phim (trung bình cộng của 2 cột "Tomatometer score" và "Audience score")

In [71]:
movie['Score'] = (movie['Tomatometer score'] + movie['Audience score'])/2

In [72]:
movie.drop(columns=['Tomatometer score', 'Audience score'], inplace=True)

## Tính số lượt review

In [87]:
def get_tomatometer_count(val):
    return int(val.split(' ')[0])

In [88]:
movie['Tomatometer count'] = movie['Tomatometer count'].apply(get_tomatometer_count)

In [90]:
movie['Audience count'].unique()

array(['1,000+ Ratings', 'Fewer than 50 Ratings', '2,500+ Ratings',
       '50,000+ Ratings', '500+ Ratings', '5,000+ Ratings',
       '100,000+ Ratings', '100+ Ratings', '25,000+ Ratings',
       '50+ Ratings', '250,000+ Ratings', '10,000+ Ratings',
       '250+ Ratings', '1,000+ Verified Ratings', '0 Ratings',
       '2,500+ Verified Ratings', '10,000+ Verified Ratings',
       '100+ Verified Ratings', '5,000+ Verified Ratings',
       '50+ Verified Ratings', 'Fewer than 50 Verified Ratings',
       '500+ Verified Ratings', '250+ Verified Ratings',
       '50,000+ Verified Ratings', '25,000+ Verified Ratings', 'Ratings'],
      dtype=object)

In [97]:
def get_audience_count(val):
    if '+' in val:
        kq = int(val.split('+')[0].replace(',', ''))
    else:
        kq = 50
    return kq

In [98]:
movie['Audience count'] = movie['Audience count'].apply(get_audience_count)

In [100]:
movie['Review count'] = movie['Tomatometer count'] + movie['Audience count']

In [103]:
movie.drop(columns=['Tomatometer count', 'Audience count'], inplace=True)

# Lưu dữ liệu đã qua tiền xử lí thành file csv

In [106]:
# movie.to_csv('movie_preprocessing.csv', index=False)